# Auto Grading Example

This can be put in a loop to grade for an entire class. Use the `reload` module if necessary for re-importing other modules. 

In [1]:
import pandas as pd
import numpy as np
import os
import sys
from IPython.display import clear_output
from pylatexenc.latexencode import unicode_to_latex
from latex_utils import make_preamble, end
from grading_utils import highlight_wrong, test_function, programming_output

# Student Input and Grading

In [2]:
# MC Solutions
mc_solutions = np.random.choice(list('abcd'), size = 10)
mc_sol_series = pd.Series(mc_solutions)
mc_sol_series.name = 'solutions'

In [3]:
# Grading

## Multiple choice

# start body for latex document
body = r"""
\section{Multiple Choice}
"""

student_mc = pd.read_csv("student_answers.csv")
student_mc = student_mc.join(mc_sol_series)
student_mc.index.name = 'question'

n_correct = (student_mc['student_answer'] == mc_sol_series).sum()
n = len(mc_sol_series)

# make column strings latex-safe
student_mc.columns = [unicode_to_latex(s) for s in student_mc.columns]

# highlight incorrect answers in red
styled_student_mc = student_mc.style.apply(highlight_wrong, props='cellcolor:{red!30}', 
                  solutions = mc_sol_series, axis=0, subset = ['student\\_answer'])

# add table and score to document body
body += styled_student_mc.to_latex(hrules = True)
body += fr"""\smallskip

\noindent Correct: {n_correct} of {n}"""


## Programming

body += r"""
\section{Programming}
\subsection*{Question 0}
"""

### Question 0
from function0 import addition as temp
from function0_answer import addition as temp_answer
test_inputs = [(1,2), (-1,1), (0,0), (10**2, 0)]

# add testing results and print submission and solution code
body = programming_output(temp, temp_answer, test_inputs, body)

### Question 1
body += r"""
\subsection*{Question 1}
"""

from function1 import prime_floor as temp
from function1_answer import prime_floor as temp_answer
test_inputs = np.random.choice(range(3,1000), size = 4)

# add testing results and print submission and solution code
body = programming_output(temp, temp_answer, test_inputs, body)

# LaTeX 

In [4]:
student = 'William Gosset'
preamble = make_preamble(student)
doc = preamble + body + end

In [5]:
student2 = student.replace(" ",'_').lower()
filename = f'{student2}_exam_report'

with open(filename + ".tex", 'w') as tf:
    tf.write(doc)

#os.system("open " + filename + ".tex")
os.system("pdflatex " + filename + ".tex")
os.system("pdflatex " + filename + ".tex")
for suffix in ['.aux','.log']:
    os.system("rm " + filename + suffix)
os.system("open " + filename + ".pdf")

clear_output()